# Short Version Extracting 30-year Running Averages Scenario - Mean Values

In [1]:
##########################################################
#
# Library Calls.
#

# loading numpy

import numpy      as np
import xarray     as xr
import zarr       as zarr
import dask       as da
import h5netcdf   as h5netcdf
import datetime   as datetime
import os         as os
import subprocess as subprocess
import dask       as dask

try:
    get_ipython()
    am_iPython = True
    print("Code is running in a Jupyter/IPython environment.")
except NameError:
    am_iPython = False
    print("Code is running in a standard Python environment (CLI/script).")

def showme(x):
    if am_iPython:
        display(x)
    else:
        print(x)
#
##########################################################

Code is running in a Jupyter/IPython environment.


In [2]:
##########################################################
#
# Pulling Surface Mask
#

print("")
print("###########################")
print("Pulling Surface Mask")

ds_mask = xr.open_dataset("https://thredds.ias.sdsmt.edu:8443/thredds/dodsC/LOCA2/loca2_mask.nc")
mask    = ds_mask["loca2_mask"]

showme(mask)

print("###########################")
print("")

#
##########################################################


###########################
Pulling Surface Mask


<xarray.DataArray 'loca2_mask' (lat: 474, lon: 944)> Size: 2MB
[447456 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 2kB 23.91 23.97 24.03 24.09 ... 53.34 53.41 53.47
  * lon      (lon) float32 4kB 234.5 234.6 234.7 234.7 ... 293.3 293.4 293.5
Attributes:
    description:    mask
    long_name:      mask
    standard_name:  land_binary_mask
    units:          1

###########################



In [3]:
##########################################################
#
# Creating Scenario File Arrays
#

print("")
print("###########################")
print("Creating Scenario File Arrays")

original_ens_file = "./LOCA2_Running_1990-2050-2084_CONUS.nc"

ds = xr.open_dataset(filename_or_obj = original_ens_file)

climatology_bounds = ds["climatology_bounds"]

print("masking")

ds = ds.where(mask == 1)

ds["climatology_bounds"] = climatology_bounds

showme(ds)

print("###########################")
print("")

#
##########################################################


###########################
Creating Scenario File Arrays
masking


<xarray.Dataset> Size: 54GB
Dimensions:             (year: 3, bnds: 2, model_member: 26, scenario: 4,
                         month: 12, lat: 474, lon: 944)
Coordinates:
  * year                (year) float32 12B 1.99e+03 2.05e+03 2.084e+03
  * model_member        (model_member) int16 52B 1 2 3 4 5 6 ... 22 23 24 25 26
  * scenario            (scenario) <U10 160B 'historical' 'ssp245' ... 'ssp585'
  * month               (month) int16 24B 1 2 3 4 5 6 7 8 9 10 11 12
  * lat                 (lat) float32 2kB 23.91 23.97 24.03 ... 53.41 53.47
  * lon                 (lon) float32 4kB 234.5 234.6 234.7 ... 293.4 293.5
Dimensions without coordinates: bnds
Data variables:
    climatology_bounds  (year, bnds) int32 24B ...
    pr                  (year, model_member, scenario, month, lat, lon) float64 13GB ...
    tasavg              (year, model_member, scenario, month, lat, lon) float64 13GB ...
    tasmax              (year, model_member, scenario, month, lat, lon) float64 13GB ...
    tasmin              (year, model_member, scenario, month, lat, lon) float64 13GB ...

###########################



In [4]:
##########################################################
#
# Process Ensemble Means
#

print("")
print("###########################")
print("Process Ensemble Means")

ds_ensmean = ds.mean(dim        = "model_member", 
                     skipna     = True, 
                     keep_attrs = True)

ds_ensmean["climatology_bounds"]              = climatology_bounds
ds_ensmean["model_member"]                    = ds["model_member"]

ds_ensmean["pr"].attrs["long_name"]           = "Ensemble Mean Total Monthly Precipitation" 
ds_ensmean["pr"].attrs["description"]         = "Ensemble Mean Total Monthly Precipitation"
ds_ensmean["pr"].attrs["cell_methods"]        = "time: sum within days  time: mean over months  model_member: mean"

ds_ensmean["tasmax"].attrs["long_name"]       = "Ensemble Mean Monthly 2-m Daily Max Temperature"
ds_ensmean["tasmax"].attrs["description"]     = "Ensemble Mean Monthly 2-m Daily Max Temperature"
ds_ensmean["tasmax"].attrs["cell_methods"]    = "time: maximum within days  time: mean over months  model_member: mean"

ds_ensmean["tasmin"].attrs["long_name"]       = "Ensemble Mean Monthly 2-m Daily Min Temperature"
ds_ensmean["tasmin"].attrs["description"]     = "Ensemble Mean Monthly 2-m Daily Min Temperature"
ds_ensmean["tasmin"].attrs["cell_methods"]    = "time: minimum within days  time: mean over months  model_member: mean"

ds_ensmean["tasavg"].attrs["long_name"]       = "Ensemble Mean Monthly 2-m Daily Temperature"
ds_ensmean["tasavg"].attrs["description"]     = "Ensemble Mean Monthly 2-m Daily Temperature"
ds_ensmean["tasavg"].attrs["cell_methods"]    = "time: mean within days  time: mean over months  model_member: mean"

showme(ds_ensmean)

print("###########################")
print("")

#
##########################################################


###########################
Process Ensemble Means


<xarray.Dataset> Size: 2GB
Dimensions:             (year: 3, bnds: 2, scenario: 4, month: 12, lat: 474,
                         lon: 944, model_member: 26)
Coordinates:
  * year                (year) float32 12B 1.99e+03 2.05e+03 2.084e+03
  * scenario            (scenario) <U10 160B 'historical' 'ssp245' ... 'ssp585'
  * month               (month) int16 24B 1 2 3 4 5 6 7 8 9 10 11 12
  * lat                 (lat) float32 2kB 23.91 23.97 24.03 ... 53.41 53.47
  * lon                 (lon) float32 4kB 234.5 234.6 234.7 ... 293.4 293.5
  * model_member        (model_member) int16 52B 1 2 3 4 5 6 ... 22 23 24 25 26
Dimensions without coordinates: bnds
Data variables:
    climatology_bounds  (year, bnds) int32 24B ...
    pr                  (year, scenario, month, lat, lon) float64 515MB nan ....
    tasavg              (year, scenario, month, lat, lon) float64 515MB nan ....
    tasmax              (year, scenario, month, lat, lon) float64 515MB nan ....
    tasmin              (year, scenario, month, lat, lon) float64 515MB nan ....

###########################



In [5]:
##########################################################
#
# Process Ensemble Means
#

print("")
print("###########################")
print("Process Ensemble Standard Deviations")

ds_ensstd = ds.std(dim        = "model_member", 
                   ddof       =    1, 
                   skipna     = True, 
                   keep_attrs = True)

print("Renaming Variable Names")

ds_ensstd = ds_ensstd.rename_vars(name_dict = {"pr"    :     "pr_sd",
                                               "tasmin": "tasmin_sd",
                                               "tasavg": "tasavg_sd",
                                               "tasmax": "tasmax_sd"})

ds_ensmean["climatology_bounds"]             = climatology_bounds
ds_ensstd["model_member"]                    = ds["model_member"]

ds_ensstd["pr_sd"].attrs["long_name"]        = "Ensemble Standard Deviation of Total Monthly Precipitation"
ds_ensstd["pr_sd"].attrs["description"]      = "Ensemble Standard Deviation of Total Monthly Precipitation"
ds_ensstd["pr_sd"].attrs["cell_methods"]     = "time: sum within days  time: mean over months  model_member: standard_deviation"

ds_ensstd["tasmax_sd"].attrs["long_name"]    = "Ensemble Standard Deviation of Mean Monthly 2-m Daily Max Temperature"
ds_ensstd["tasmax_sd"].attrs["description"]  = "Ensemble Standard Deviation of Mean Monthly 2-m Daily Max Temperature"
ds_ensstd["tasmax_sd"].attrs["cell_methods"] = "time: maximum within days  time: mean over months  model_member: standard_deviation"

ds_ensstd["tasmin_sd"].attrs["long_name"]    = "Ensemble Standard Deviation of Mean Monthly 2-m Daily Min Temperature"
ds_ensstd["tasmin_sd"].attrs["description"]  = "Ensemble Standard Deviation of Mean Monthly 2-m Daily Min Temperature"
ds_ensstd["tasmin_sd"].attrs["cell_methods"] = "time: minimum within days  time: mean over months  model_member: standard_deviation"

ds_ensstd["tasavg_sd"].attrs["long_name"]    = "Ensemble Standard Deviation of Mean Monthly 2-m Daily Temperature"
ds_ensstd["tasavg_sd"].attrs["description"]  = "Ensemble Standard Deviation of Mean Monthly 2-m Daily Temperature"
ds_ensstd["tasavg_sd"].attrs["cell_methods"] = "time: mean within days  time: mean over months  model_member: standard_deviation"

showme(ds_ensstd)

print("---------------------------")
print("sd tasmin ",ds_ensstd["tasmin_sd"].max(skipna = True).values)
print("sd tasavg ",ds_ensstd["tasavg_sd"].max(skipna = True).values)
print("sd tasmax ",ds_ensstd["tasmax_sd"].max(skipna = True).values)
print("sd tasmax ",ds_ensstd["tasmax_sd"].max(skipna = True).values)

print("###########################")
print("")

#
##########################################################


###########################
Process Ensemble Standard Deviations


/Users/wjc/miniforge3/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:4268: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/wjc/miniforge3/lib/python3.12/site-packages/numpy/_core/_methods.py:211: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


Renaming Variable Names


<xarray.Dataset> Size: 2GB
Dimensions:             (year: 3, bnds: 2, scenario: 4, month: 12, lat: 474,
                         lon: 944, model_member: 26)
Coordinates:
  * year                (year) float32 12B 1.99e+03 2.05e+03 2.084e+03
  * scenario            (scenario) <U10 160B 'historical' 'ssp245' ... 'ssp585'
  * month               (month) int16 24B 1 2 3 4 5 6 7 8 9 10 11 12
  * lat                 (lat) float32 2kB 23.91 23.97 24.03 ... 53.41 53.47
  * lon                 (lon) float32 4kB 234.5 234.6 234.7 ... 293.4 293.5
  * model_member        (model_member) int16 52B 1 2 3 4 5 6 ... 22 23 24 25 26
Dimensions without coordinates: bnds
Data variables:
    climatology_bounds  (year, bnds) float64 48B nan nan nan nan nan nan
    pr_sd               (year, scenario, month, lat, lon) float64 515MB nan ....
    tasavg_sd           (year, scenario, month, lat, lon) float64 515MB nan ....
    tasmax_sd           (year, scenario, month, lat, lon) float64 515MB nan ....
    tasmin_sd           (year, scenario, month, lat, lon) float64 515MB nan ....

---------------------------
sd tasmin  3.6955639600007117
sd tasavg  3.2707278729227003
sd tasmax  3.4519921784384167
sd tasmax  3.4519921784384167
###########################



In [6]:
##########################################################
#
# Creating Mean NetCDF File
#

print("Creating Mean NetCDF File")
chunks_tuple = (0,0,12,len(ds_ensmean["lat"]),len(ds_ensmean["lon"]))

ds_ensmean.to_netcdf(path           =   "./rolling_monthly_mean.nc", 
                     mode           =           'w', 
                     format         =     "NETCDF4",
                     engine         =    "h5netcdf",
                     unlimited_dims =        "year",
                     encoding       = {"pr"       : {        "zlib":    True,
                                                        "complevel":       7, 
                                                            "dtype": "int16", 
                                                     "scale_factor":     0.1,
                                                       "add_offset":     0.0,                                        
                                                       "_FillValue":  -32767},
                                      "tasmax"    : {        "zlib":    True,
                                                        "complevel":       7, 
                                                            "dtype": "int16", 
                                                     "scale_factor":     0.1,
                                                       "add_offset":     0.0,                                        
                                                       "_FillValue":  -32767},
                                      "tasavg"    : {        "zlib":    True,
                                                        "complevel":       7, 
                                                            "dtype": "int16", 
                                                     "scale_factor":     0.1,
                                                       "add_offset":     0.0,                                        
                                                       "_FillValue":  -32767},
                                      "tasmin"    : {        "zlib":    True,
                                                        "complevel":       7, 
                                                            "dtype": "int16", 
                                                     "scale_factor":     0.1,
                                                       "add_offset":     0.0,                                        
                                                       "_FillValue":  -32767}})


print("")
print("===========================")
print("MASTER MEAN FILE WRITTEN")
print("===========================")
print("")

print("###########################")
print("")
#
##########################################################

Creating Mean NetCDF File

MASTER MEAN FILE WRITTEN

###########################



In [7]:
##########################################################
#
# Creating Stdev NetCDF File
#

print("Creating Stdev NetCDF File")
chunks_tuple = (0,0,12,len(ds_ensstd["lat"]),len(ds_ensstd["lon"]))

ds_ensstd.to_netcdf( path           =   "./rolling_monthly_stdev.nc", 
                     mode           =           'w', 
                     format         =     "NETCDF4",
                     engine         =    "h5netcdf",
                     unlimited_dims =        "year",
                     encoding      = {"pr_sd"     : {        "zlib":    True,
                                                        "complevel":       7, 
                                                            "dtype": "int16", 
                                                     "scale_factor":     0.01,
                                                       "add_offset":     0.0,                                        
                                                       "_FillValue":  -32767},
                                      "tasmax_sd" : {        "zlib":    True,
                                                        "complevel":       7, 
                                                            "dtype": "int16", 
                                                     "scale_factor":    0.01,
                                                       "add_offset":     0.0,                                        
                                                       "_FillValue":  -32767},
                                      "tasavg_sd" : {        "zlib":    True,
                                                        "complevel":       7, 
                                                            "dtype": "int16", 
                                                     "scale_factor":    0.01,
                                                       "add_offset":     0.0,                                        
                                                       "_FillValue":  -32767},
                                      "tasmin_sd" : {        "zlib":    True,
                                                        "complevel":       7, 
                                                            "dtype": "int16", 
                                                     "scale_factor":    0.01,
                                                       "add_offset":     0.0,                                        
                                                       "_FillValue":  -32767}})


print("")
print("===========================")
print("MASTER STDEV FILE WRITTEN")
print("===========================")
print("")

print("###########################")
print("")
#
##########################################################

Creating Stdev NetCDF File

MASTER STDEV FILE WRITTEN

###########################



In [8]:
##########################################################
#
# Closeout
#

print("")
print("###########################")
print("DONE!")
print("###########################")
print("")
#
##########################################################


###########################
DONE!
###########################

